In [ ]:
# !pip uninstall tensorflow -y --quiet
# !pip install tensorflow==2.15 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.71 requires ml_dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.


In [ ]:
# mlp2.py
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Normalization
from tensorflow.keras import optimizers
import numpy as np; import sys

(AX, AY), (QX, QY) = mnist.load_data()
AX=255-AX; QX=255-QX

nclasses = 10
AY2 = keras.utils.to_categorical(AY, nclasses)
QY2 = keras.utils.to_categorical(QY, nclasses)

nl, nc = AX.shape[1], AX.shape[2] #28, 28

model = Sequential()
model.add(Normalization(input_shape=(nl,nc))) #Normaliza
model.add(Flatten())
model.add(Dense(400, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(nclasses, activation='softmax'))

opt=optimizers.Adam(learning_rate=0.0005)
# como usou a softmax teve que mudar a função de erro para categorical_crossentropy
model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy'])
model.get_layer(index=0).adapt(AX) #Calcula media e desvio
model.fit(AX, AY2, batch_size=100, epochs=80, verbose=2);

score = model.evaluate(QX, QY2, verbose=False)
print('Test loss: %.4f'%(score[0]))
print('Test accuracy: %.2f %%'%(100*score[1]))
print('Test error: %.2f %%'%(100*(1-score[1])))
model.save('mlp2.keras')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/80
600/600 - 5s - 8ms/step - accuracy: 0.9267 - loss: 0.2502
Epoch 2/80
600/600 - 1s - 2ms/step - accuracy: 0.9723 - loss: 0.0946
Epoch 3/80
600/600 - 1s - 2ms/step - accuracy: 0.9834 - loss: 0.0564
Epoch 4/80
600/600 - 1s - 2ms/step - accuracy: 0.9888 - loss: 0.0374
Epoch 5/80
600/600 - 1s - 2ms/step - accuracy: 0.9923 - loss: 0.0260
Epoch 6/80
600/600 - 1s - 2ms/step - accuracy: 0.9944 - loss: 0.0207
Epoch 7/80
600/600 - 1s - 2ms/step - accuracy: 0.9943 - loss: 0.0203
Epoch 8/80
600/600 - 1s - 2ms/step - accuracy: 0.9959 - loss: 0.0134
Epoch 9/80
600/600 - 1s - 2ms/step - accuracy: 0.9959 - loss: 0.0136
Epoch 10/80
600/600 - 1s - 2ms/step - accuracy: 0.9979 - loss: 0.0075
Epoch 11/80
600/600 - 2s - 3ms/step - accuracy: 0.9974 - loss: 0.0082
Epoch 12/80
600/600 - 1s - 2ms/step - accuracy: 0.9956 - loss: 0.0131
Epoch 13/80
600/600 - 1s - 2ms/step - accuracy: 0.9985 - loss: 0.0054
Epoch 14/80
600/600 - 1s - 2ms/step - accuracy: 0.9986 - loss: 0.0045
Epoch 15/80
600/600 - 1s - 2m

In [ ]:
#pred2.py
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

(_,_), (QX, QY) = mnist.load_data()
QX=255-QX

nclasses = 10
QY2 = keras.utils.to_categorical(QY, nclasses)
nl, nc = QX.shape[1], QX.shape[2] #28, 28

model=load_model('mlp2.keras')
score = model.evaluate(QX, QY2, verbose=False)
print('Test loss: %.4f'%(score[0]))
print('Test accuracy: %.2f %%'%(100*score[1]))
print('Test error: %.2f %%'%(100*(1-score[1])))

QP2 = model.predict(QX); QP = QP2.argmax(axis=1)
print(QP2[0])
print(QP[0])
#Aqui QP contem as 10000 predicoes
print("Imagem-teste 0: Rotulo verdadeiro=%d, predicao=%d"%(QY[0],QP[0]))
print("Imagem-teste 1: Rotulo verdadeiro=%d, predicao=%d"%(QY[1],QP[1]))

Test loss: 0.2140
Test accuracy: 97.90 %
Test error: 2.10 %
313/313 [==============================] - 1s 2ms/step
[1.2184994e-38 1.3615631e-27 4.6658247e-29 7.7322288e-18 8.8239140e-26
 1.5799965e-33 0.0000000e+00 1.0000000e+00 6.7217944e-27 4.4224917e-20]
7
Imagem-teste 0: Rotulo verdadeiro=7, predicao=7
Imagem-teste 1: Rotulo verdadeiro=2, predicao=2
